<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Librerías" data-toc-modified-id="Librerías-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Librerías</a></span></li><li><span><a href="#Extracción-de-la-información-de-la-tabla-principal" data-toc-modified-id="Extracción-de-la-información-de-la-tabla-principal-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extracción de la información de la tabla principal</a></span></li><li><span><a href="#Extracción-de-la-información-de-los-detalles-de-cada-alerta" data-toc-modified-id="Extracción-de-la-información-de-los-detalles-de-cada-alerta-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extracción de la información de los detalles de cada alerta</a></span></li><li><span><a href="#Definición-del-cabecero" data-toc-modified-id="Definición-del-cabecero-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Definición del cabecero</a></span></li><li><span><a href="#Código-principal" data-toc-modified-id="Código-principal-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Código principal</a></span></li></ul></div>

## Librerías 

In [1]:
from selenium import webdriver 
import csv   
from selenium.common.exceptions import TimeoutException
import time

## Extracción de la información de la tabla principal
En esta función se extrae la información de la tabla principal, es decir, de la tabla que es mostrada al hacer una búsqueda vacía en RASFF portal(https://webgate.ec.europa.eu/rasff-window/portal/?event=searchResultList).

El programa principal ya envía a esta función la alerta de la que hay que sacar la información, por lo que esta función tan solo extrae la información de dicha alerta y llama a la siguiente función para extraer la información de los detalles.

In [2]:
def principal_info(alert):

    alert_info = []
    csv_row = []
    count = 1
    code = ""
    attributes = alert.find_elements_by_xpath("td") #Extraemos los elementos (columnas) de la alerta.

    for i in attributes:                        #Recorremos los elementos de la alerta y los añadimos a la variable line de tipo
        if count == 10:                         #lista, que va a contener toda la información de cada alerta.               
            break
        if count == 4:                          #El cuarto elemento (count ==4) corresponde con la referencia, que guardaremos
            code = i.text                       #para poder indicar a la siguiente función en qué alerta nos encontramos. 
        alert_info.append(i.text)                  
        count += 1
                                               
    details_info(code,alert_info)               #Llamada a la función que extrae los detalles.
    csv_row.append(alert_info)
    
    
    alert_number = csv_row[0][0]
    alert_number = int(alert_number[:-1])
    if int(alert_number) % 100 == 0:
        print(alert_number)
    
    return csv_row

In [3]:
def details_anomalies(test,code):
    for t in test:
        if 'Notification details - ' + str(code) in t.text:
            return False
        
    return True

In [4]:
def main_anomalies(test,first_row):
    for t in test:
        if 'Notifications ' + str(first_row) in t.text:
            return False
    
    return True

## Extracción de la información de los detalles de cada alerta
En esta función se extrae la información contenida en la página de detalles de la alerta en cuestión. En primer lugar, se extrae la información del cuerpo principal que no hemos extraido en la función anterior (hay información aquí contenida que ya ha sido extraida en la tabla principal). Después, se extrae la información más detallada, como los países que se ven relacionados de alguna manera con la alerta.


In [5]:
def details_info(code,alert_info):
    try:
        count = 1
        url_details = "https://webgate.ec.europa.eu/rasff-window/portal/?event=notificationDetail&NOTIF_REFERENCE="+code
        browser_details = webdriver.Firefox()
        
        browser_details.get(url_details)         #Accedemos a la página que contiene los detalles de la alerta. 
        test = browser_details.find_elements_by_xpath("//div")
        anomalies = details_anomalies(test,code)
        
        while anomalies == True:
            print("ANOMALIA DETECTADA")
            time.sleep(120)
            
            browser_details.get(url_details)
            test = browser_details.find_elements_by_xpath("//div")
            anomalies = details_anomalies(test,code)
            
        details = browser_details.find_elements_by_xpath("//div[@class='span6']")#Extraemos todos los elementos del cuerpo principal.
        
        for i in details:                        #Recorremos los elementos extraidos e incorporamos solo aquellos que no tenemos ya.      
            if count == 2 :
                alert_info.append(i.text)
            if count == 3 :
                alert_info.append(i.text)
            if count == 4 :
                alert_info.append(i.text)
            count += 1
            
        
        num_hazards = 0
        hazards = ""
        hazardscat = ""
        
        all_hazards = browser_details.find_elements_by_xpath("//table[@id='hazards']/tbody[1]/tr")
        
        for i in all_hazards:
            num_hazards = num_hazards +1


        for i in range (1,num_hazards+1):
            
            var = str(i)
    
            hazard = browser_details.find_element_by_xpath("//table[@id='hazards']/tbody[1]/tr["+var+"]/td[1]")
            hazardcat = browser_details.find_element_by_xpath("//table[@id='hazards']/tbody[1]/tr["+var+"]/td[2]")
            
            if i != 1:
                hazards = hazards + ","
                hazardscat = hazardscat + ","
            hazards = hazards + hazard.text
            hazardscat = hazardscat + hazardcat.text
        
        
        alert_info.append(hazards)
        alert_info.append(hazardscat)    

        ''' A continuación, extraemos todos los países involucrados en la alerta clasificándolos en países de origen, países de
        destino y otros países. Esto lo hacemos identificando algunos símbolos que llevan en su nombre y que los identifican. Ha
        habido que añadir varias excepciones debido a que que a parte de las etiquetas básicas de origen "(O)" y destino "(D)" 
        existen otras como origen y destino "(O/D)" e incluso algunos paises constituyen links hacia otra información, por lo
        que se encuentran en otro nodo del árbol de la página web y es necesaria una búsqueda diferente para encontrarlos.'''
        ori_list = ""
        des_list = ""
        oth_list = ""
        wrong = 0
        origin1 = browser_details.find_elements_by_xpath("//span[contains(text(),'(O)') and @style='width: auto; color: rgb(51, 51, 51); background-color: rgb(238, 238, 238);'] | //span[contains(text(),'(D/O)') and @style='width: auto; color: rgb(51, 51, 51); background-color: rgb(238, 238, 238);']")
        origin2 = browser_details.find_elements_by_partial_link_text('(O)')
        origin3 = browser_details.find_elements_by_partial_link_text('(D/O)')
        origins = origin1 + origin2 + origin3
        destination1 = browser_details.find_elements_by_xpath("//span[contains(text(),'(D)') and @style='width: auto; color: rgb(51, 51, 51); background-color: rgb(238, 238, 238);'] | //span[contains(text(),'(D/O)') and @style='width: auto; color: rgb(51, 51, 51); background-color: rgb(238, 238, 238);']")
        destination2 = browser_details.find_elements_by_partial_link_text('(D)')
        destination3 = browser_details.find_elements_by_partial_link_text('(D/O)')
        destinations = destination1 + destination2 + destination3
        other1 = browser_details.find_elements_by_xpath("//span[not(contains(text(),'(D)')) and not(contains(text(), '(O)')) and @style='width: auto; color: rgb(51, 51, 51); background-color: rgb(238, 238, 238);']")
        other2 = browser_details.find_elements_by_xpath("//div[@class = 'list']/span/a")
        others = other1 + other2

        count = 1
        for i in origins:                  #Recorremos los paises de origen y añadimos una "," entre cada uno de ellos.
            if count != 1:
                ori_list = ori_list + ","
            long = 0
            orig = ""
            orig = i.text
            for a in i.text:
                long += 1
                if a == '(':
                    orig = i.text[:long-2] #Identificamos el símbolo "(" y cortamos a partir de ahí, pues ya no necesitamos el 
                    break                  #identificador.
            ori_list = ori_list + orig
            count += 1 
        alert_info.append(ori_list)        #Añadimos todos los países de origen a la variable que contiene la información de toda 
                                           #la alerta.
        count = 1
        for i in destinations:             #Recorremos los paises de destino y añadimos una "," entre cada uno de ellos.
            if count != 1:
                des_list = des_list + ","
            long = 0
            dest = ""
            dest = i.text
            for a in i.text:
                long += 1
                if a == '(':
                    dest = i.text[:long-2] #Identificamos el símbolo "(" y cortamos a partir de ahí, pues ya no necesitamos el
                    break                  #identificador.
            des_list = des_list + dest
            count += 1         
        alert_info.append(des_list)        #Añadimos todos los países de destino a la variable que contiene la información de toda 
                                           #la alerta.
        count = 1
        for i in others:                   #Recorremos el resto de paises. En este grupo se cuelan algunos paises que ya hemos
            wrong = 0                      #añadido debido a la estructura de la página, por lo que los identificamos por el símbolo 
            for a in i.text:               # "(" (los paises que deben ir en este grupo no llevan ningún identificador) y no los 
                if a == '(':               #añadimos
                    wrong = 1
                    break
            if wrong == 0:
                if count != 1:
                    oth_list = oth_list + ","
                oth = ""
                oth = i.text[:-1]
                oth_list = oth_list + oth
                count += 1 
        alert_info.append(oth_list)        #Añadimos todos los países restantes a la variable que contiene la información de toda
                                           #la alerta
        browser_details.close()            #Cerramos la pestaña de detalles
    
    except TimeoutException:
        browser_details.implicitly_wait(180)
        browser_details.close()  
        
    return

## Definición del cabecero
Esta función define el cabecero de la tabla que vamos a generar con todos los datos extraidos.

In [6]:
def header():
    csv_row=[]
    head = ["NUMBER", "CLASSIF", "DATE_CASE", "REF", "NOT_COUNTRY", "SUBJET", "PROD_CAT", "TYPE", "RISK_DECISION", "ACTION_TAKEN", "DISTRIBUTION_STAT", "PRODUCT", "HAZARDS", "HAZARDS_CAT", "COUNT_ORIGEN", "COUNT_DESTIN", "COUNT_CONCERN"]
    csv_row.append(head)
    return csv_row

## Código principal 
La función principal de esta parte es iniciar el navegador, hacer una búsqueda vacía para que se nos muestre todos los resultados y recorrer cada alerta.
La primera celda solo debe ser ejecutada una vez antes de extraer la información, y se debe esperar a que el navegador esté completamente cargado.
La segunda celda recorre las alertas y abre el csv en el que va a ser escrita toda la información. Así mismo, una vez que las funciones han devuelto la información extraida, se escribe en el csv.
Por último, se recorren las paginas principales que contienen las alertas (cada página contiene 100 alertas) modificando el url de la página  en la que se realiza la búsqueda principal, debido a que el propio url muestra el bloque de alertas que esta mostrando mediante un número.

In [11]:
main_url = "https://webgate.ec.europa.eu/rasff-window/portal/?event=SearchForm&cleanSearch=1"
browser_main = webdriver.Firefox()
browser_main.get(main_url)
search = browser_main.find_element_by_id("Btn_Search")
search.click()

In [12]:
row = 0
first_row = 1
url = 'https://webgate.ec.europa.eu/rasff-window/portal/?event=searchResultList'

while first_row < 56000:                                      #Recorremos cada página que contiene 100 alertas, hasta el final.
    try: 
        
        test = browser_main.find_elements_by_xpath("//div")
        anomalies = main_anomalies(test,first_row)
        
        '''while anomalies == True:                               
            print("ANOMALIA DETECTADA")
            time.sleep(120)
            
            browser_main.get(nexturl) 
            test = browser_main.find_elements_by_xpath("//div")
            anomalies = main_anomalies(test,first_row)'''
            
        alerts = browser_main.find_elements_by_xpath("//tr[@class='even']")     #Extraemos las alertas de la página principal
        alerts = alerts + browser_main.find_elements_by_xpath("//tr[@class='odd']") # Extraemos primero las alertas impares y
        
        for i in alerts:                                                            #después las pares debido a que así son más
            with open('RASFF_DATA.csv', 'a', encoding="utf-8") as f:                #fáciles de identificar.
                writer = csv.writer(f,delimiter='\t')
                if row == 0 :
                    writer.writerows(header())                                      #Escribimos la información en el csv
                writer.writerows(principal_info(i))
            row += 1
         
        first_row += 100
        nexturl = url + '&StartRow=' + str(first_row)                           #Recorremos las páginas de la búsqueda principal 
        browser_main.get(nexturl)                                               #modificando el url.                              
    
    except TimeoutException:
        browser_main.implicitly_wait(180)
        
browser_main.close()

100
25500
